In [1]:
import pandas as pd
import numpy as np
import mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
# Base com as transações
df=pd.read_csv(r"C:/Arthurtamura/Tera Dados/Projetos/insta/instacart/order_products__train/order_products__train.csv")

#Base com os nomes dos produtos
prod=pd.read_csv(r"C:/Arthurtamura/Tera Dados/Projetos/insta/instacart/products/products.csv")

df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 4 columns):
 #   Column             Non-Null Count    Dtype
---  ------             --------------    -----
 0   order_id           1384617 non-null  int64
 1   product_id         1384617 non-null  int64
 2   add_to_cart_order  1384617 non-null  int64
 3   reordered          1384617 non-null  int64
dtypes: int64(4)
memory usage: 42.3 MB


In [4]:
df.order_id.isna().sum()

0

In [5]:
df.product_id.isna().sum()

0

In [6]:
#Verificando a quantidade de Ids únicos
df.order_id.nunique()

131209

In [7]:
df.product_id.nunique()

39123

In [8]:
df.order_id.value_counts()

1395075    80
2813632    80
949182     77
2869702    76
341238     76
           ..
1144944     1
1144765     1
1144608     1
1144038     1
3214874     1
Name: order_id, Length: 131209, dtype: int64

In [9]:
df.product_id.value_counts()

24852    18726
13176    15480
21137    10894
21903     9784
47626     8135
         ...  
42744        1
5871         1
47237        1
9305         1
38900        1
Name: product_id, Length: 39123, dtype: int64

Transformação das Bases de Dados



In [24]:
# Filtrando transações com apenas um produto e produtos que apareceram poucas vezes (valor arbitrário)
# Filtro para produtos aplicado porque o algoritmo não rodou com todos, não sei como melhorar
filtro_orderId=list(df.order_id.value_counts()[df.order_id.value_counts()>1].index)
filtro_produtos=list(df.product_id.value_counts()[df.product_id.value_counts()>1000].index)

df=df[df['order_id'].isin(filtro_orderId)]
df=df[df['product_id'].isin(filtro_produtos)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 362261 entries, 2 to 1384614
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   order_id           362261 non-null  int64
 1   product_id         362261 non-null  int64
 2   add_to_cart_order  362261 non-null  int64
 3   reordered          362261 non-null  int64
dtypes: int64(4)
memory usage: 13.8 MB


In [25]:
df.order_id.nunique()

93131

In [26]:
df.product_id.nunique()


146

In [27]:
prod.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [28]:
data=df.merge(prod,left_on="product_id",right_on="product_id")
data.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,1,10246,3,0,Organic Celery Hearts,83,4
1,2869,10246,4,1,Organic Celery Hearts,83,4
2,3378,10246,19,0,Organic Celery Hearts,83,4
3,14119,10246,6,0,Organic Celery Hearts,83,4
4,17152,10246,22,1,Organic Celery Hearts,83,4


In [29]:
# Mantendo apenas order_id e product_name
data=data.iloc[:,[0,4]]

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 362261 entries, 0 to 362260
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   order_id      362261 non-null  int64 
 1   product_name  362261 non-null  object
dtypes: int64(1), object(1)
memory usage: 8.3+ MB


In [38]:
# Ao rodas as regras, verifiquei que algumas expressões estavam atrapalhando, corrigi na mão
# Talvez isso mereça maior atenção para o desempenho do modelo
substituicao={'Organic ':'','Box of ':'','Bag of':'','Bananas':'Banana','Hass':''}
data['product_name']=data['product_name'].replace(substituicao,regex=True).str.strip()
data.head(200)

,order_id,product_name
0,1,Celery Hearts
1,2869,Celery Hearts
2,3378,Celery Hearts
3,14119,Celery Hearts
4,17152,Celery Hearts
...,...,...
195,588768,Celery Hearts
196,590948,Celery Hearts
197,591133,Celery Hearts
198,591165,Celery Hearts


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 756896 entries, 0 to 756895
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   order_id      756896 non-null  int64 
 1   product_name  756896 non-null  object
dtypes: int64(1), object(1)
memory usage: 17.3+ MB


In [32]:
basket= data[['order_id','product_name']].value_counts().unstack().reset_index().fillna(0).set_index('order_id')
basket.head()

product_name,100% Raw Coconut Water,100% Recycled Paper Towels,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Avocado,Avocados,Baby Arugula,Baby Broccoli,...,Unsweetened Vanilla Almond Milk,Vanilla Almond Breeze Almond Milk,Whipped Cream Cheese,Whole Milk,Whole Strawberries,Whole String Cheese,Yellow Bell Pepper,Yellow Onion,Yellow Onions,Zucchini
order_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
36,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [33]:
def encode_units(x):
    if x <= 0:
        return False
    if x >= 1:
        return True

basket_sets = basket.applymap(encode_units)

In [34]:
basket_sets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93131 entries, 1 to 3421070
Columns: 130 entries, 100% Raw Coconut Water to Zucchini
dtypes: bool(130)
memory usage: 12.3 MB


Criação das Regras
Now that the data is structured properly, we can generate frequent item sets that have a support of at least 1% (this number was chosen so that I could get enough useful examples). These values are mostly just arbitrarily chosen, so you can play with these values and see what difference it makes in the rules you get back out.

Idea for choosing support: Let's suppose that we want rules for only those items that are purchased at least 5 times a day, or 7 x 5 = 35 times in one week, since our dataset is for a one-week time period (I don't know what is our time period). The support for those items can be calculated as 35/7500 = 0.0045


In [35]:
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True,low_memory=True)
frequent_itemsets.sort_values('support',ascending=False)

,support,itemsets
12,0.383084,(Banana)
112,0.182786,(Strawberries)
6,0.173100,(Avocado)
11,0.115837,(Baby Spinach)
84,0.093030,(Raspberries)
...,...,...
195,0.010233,"(Banana, Red Vine Tomato)"
205,0.010211,"(Unsweetened Almondmilk, Banana)"
223,0.010190,"(Strawberries, Honeycrisp Apple)"
161,0.010093,"(Red Onion, Baby Spinach)"


In [48]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Banana),(100% Whole Wheat Bread),0.383084,0.024149,0.011189,0.029206,1.209440,0.001938,1.005210
1,(100% Whole Wheat Bread),(Banana),0.024149,0.383084,0.011189,0.463317,1.209440,0.001938,1.149498
2,(Apple Honeycrisp Organic),(Banana),0.023977,0.383084,0.012735,0.531124,1.386443,0.003550,1.315734
3,(Banana),(Apple Honeycrisp Organic),0.383084,0.023977,0.012735,0.033243,1.386443,0.003550,1.009584
4,(Asparagus),(Avocado),0.040996,0.173100,0.010866,0.265060,1.531253,0.003770,1.125126
...,...,...,...,...,...,...,...,...,...
251,"(Banana, Raspberries)",(Strawberries),0.044357,0.182786,0.018275,0.412007,2.254045,0.010168,1.389837
252,"(Raspberries, Strawberries)",(Banana),0.033652,0.383084,0.018275,0.543076,1.417642,0.005384,1.350150
253,(Banana),"(Raspberries, Strawberries)",0.383084,0.033652,0.018275,0.047706,1.417642,0.005384,1.014758
254,(Strawberries),"(Banana, Raspberries)",0.182786,0.044357,0.018275,0.099982,2.254045,0.010168,1.061805


A galera parece bem "fitness" com esses filtros

Confidence refers to the likelihood that an item B is also bought if item A is bought. It can be calculated by finding the number of transactions where A and B are bought together, divided by total number of transactions where A is bought. In the second rule, the confidence is 0,36, which means that 36% of times where fat milk was bought, banana is also bought.

Lift(A -> B) refers to the increase in the ratio of sale of B when A is sold. Lift(A –> B) can be calculated by dividing Confidence(A -> B) divided by Support(B). a lift of 1.29 tells us that the likelihood of buying a A and B together is 1.29 times more than the likelihood of just buying the B. (30% chance)

I don't know yet what conviction is.

Conclusion
Association rule mining algorithms such as Apriori are very useful for finding simple associations between our data items. They are easy to implement and have high explain-ability. However for more advanced insights, such those used by Google or Amazon etc., more complex algorithms, such as recommender systems, are used. However, you can probably see that this method is a very simple way to get basic associations if that's all your use-case needs.

we could try to use departments

In [100]:
# se eu aumentar a régua para 30%
frequent_itemsets = apriori(basket_sets, min_support=0.03, use_colnames=True,low_memory=True)
frequent_itemsets.sort_values('support',ascending=False)

,support,itemsets
5,0.383084,(Banana)
29,0.182786,(Strawberries)
1,0.173100,(Avocado)
4,0.115837,(Baby Spinach)
22,0.093030,(Raspberries)
41,0.090851,"(Banana, Strawberries)"
18,0.086566,(Large Lemon)
35,0.084591,"(Banana, Avocado)"
7,0.076387,(Blueberries)
30,0.065156,(Whole Milk)


In [107]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.05)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Baby Spinach),(Avocado),0.115837,0.173100,0.032975,0.284668,1.644528,0.012924,1.155967
1,(Avocado),(Baby Spinach),0.173100,0.115837,0.032975,0.190497,1.644528,0.012924,1.092229
2,(Banana),(Avocado),0.383084,0.173100,0.084591,0.220815,1.275645,0.018279,1.061236
3,(Avocado),(Banana),0.173100,0.383084,0.084591,0.488679,1.275645,0.018279,1.206515
4,(Strawberries),(Avocado),0.182786,0.173100,0.040631,0.222287,1.284155,0.008991,1.063246
5,(Avocado),(Strawberries),0.173100,0.182786,0.040631,0.234725,1.284155,0.008991,1.067870
6,(Banana),(Baby Spinach),0.383084,0.115837,0.054407,0.142024,1.226072,0.010032,1.030522
7,(Baby Spinach),(Banana),0.115837,0.383084,0.054407,0.469689,1.226072,0.010032,1.163309
8,(Banana),(Blueberries),0.383084,0.076387,0.034264,0.089441,1.170898,0.005001,1.014337
9,(Blueberries),(Banana),0.076387,0.383084,0.034264,0.448552,1.170898,0.005001,1.118720


In [111]:
rules.sort_values('lift'and 'confidence',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
15,(Strawberries),(Banana),0.182786,0.383084,0.090851,0.497033,1.297453,0.020828,1.226555
3,(Avocado),(Banana),0.173100,0.383084,0.084591,0.488679,1.275645,0.018279,1.206515
13,(Raspberries),(Banana),0.093030,0.383084,0.044357,0.476801,1.244637,0.008718,1.179122
7,(Baby Spinach),(Banana),0.115837,0.383084,0.054407,0.469689,1.226072,0.010032,1.163309
9,(Blueberries),(Banana),0.076387,0.383084,0.034264,0.448552,1.170898,0.005001,1.118720
10,(Large Lemon),(Banana),0.086566,0.383084,0.036336,0.419747,1.095705,0.003174,1.063184
16,(Raspberries),(Strawberries),0.093030,0.182786,0.033652,0.361727,1.978968,0.016647,1.280352
0,(Baby Spinach),(Avocado),0.115837,0.173100,0.032975,0.284668,1.644528,0.012924,1.155967
14,(Banana),(Strawberries),0.383084,0.182786,0.090851,0.237156,1.297453,0.020828,1.071273
5,(Avocado),(Strawberries),0.173100,0.182786,0.040631,0.234725,1.284155,0.008991,1.067870


In [103]:
rules.sort_values('confidence',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
15,(Strawberries),(Banana),0.182786,0.383084,0.090851,0.497033,1.297453,0.020828,1.226555
3,(Avocado),(Banana),0.173100,0.383084,0.084591,0.488679,1.275645,0.018279,1.206515
13,(Raspberries),(Banana),0.093030,0.383084,0.044357,0.476801,1.244637,0.008718,1.179122
7,(Baby Spinach),(Banana),0.115837,0.383084,0.054407,0.469689,1.226072,0.010032,1.163309
9,(Blueberries),(Banana),0.076387,0.383084,0.034264,0.448552,1.170898,0.005001,1.118720
10,(Large Lemon),(Banana),0.086566,0.383084,0.036336,0.419747,1.095705,0.003174,1.063184
16,(Raspberries),(Strawberries),0.093030,0.182786,0.033652,0.361727,1.978968,0.016647,1.280352
0,(Baby Spinach),(Avocado),0.115837,0.173100,0.032975,0.284668,1.644528,0.012924,1.155967
14,(Banana),(Strawberries),0.383084,0.182786,0.090851,0.237156,1.297453,0.020828,1.071273
5,(Avocado),(Strawberries),0.173100,0.182786,0.040631,0.234725,1.284155,0.008991,1.067870


In [68]:
# Vou analisar com a outra base
# Base com as transações
dz=pd.read_csv(r"C:/Arthurtamura/Tera Dados/Projetos/insta/instacart/order_products__prior/order_products__prior.csv")
dz.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [69]:
#Verificando a quantidade de Ids únicos
dz.order_id.nunique()

3214874

In [91]:
filtro_orderId=list(dz.order_id.value_counts()[dz.order_id.value_counts()>1].index)
filtro_produtos=list(dz.product_id.value_counts()[dz.product_id.value_counts()>10000].index)

dz=dz[dz['order_id'].isin(filtro_orderId)]
dz=dz[dz['product_id'].isin(filtro_produtos)]
dz.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13534176 entries, 0 to 24700790
Data columns (total 7 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   order_id           int64 
 1   product_id         int64 
 2   add_to_cart_order  int64 
 3   reordered          int64 
 4   product_name       object
 5   aisle_id           int64 
 6   department_id      int64 
dtypes: int64(6), object(1)
memory usage: 826.1+ MB


In [92]:
data2=dz.merge(prod,left_on="product_id",right_on="product_id")
data2.head(200)

,order_id,product_id,add_to_cart_order,reordered,product_name_x,aisle_id_x,department_id_x,product_name_y,aisle_id_y,department_id_y
0,2,33120,1,1,Organic Egg Whites,86,16,Organic Egg Whites,86,16
1,26,33120,5,0,Organic Egg Whites,86,16,Organic Egg Whites,86,16
2,120,33120,13,0,Organic Egg Whites,86,16,Organic Egg Whites,86,16
3,327,33120,5,1,Organic Egg Whites,86,16,Organic Egg Whites,86,16
4,390,33120,28,1,Organic Egg Whites,86,16,Organic Egg Whites,86,16
...,...,...,...,...,...,...,...,...,...,...
195,31911,33120,5,1,Organic Egg Whites,86,16,Organic Egg Whites,86,16
196,32026,33120,11,1,Organic Egg Whites,86,16,Organic Egg Whites,86,16
197,32236,33120,2,0,Organic Egg Whites,86,16,Organic Egg Whites,86,16
198,32256,33120,15,1,Organic Egg Whites,86,16,Organic Egg Whites,86,16


In [93]:
# Mantendo apenas order_id e product_name
data2=data2.iloc[:,[0,4]]

In [94]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13534176 entries, 0 to 13534175
Data columns (total 2 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   order_id        int64 
 1   product_name_x  object
dtypes: int64(1), object(1)
memory usage: 309.8+ MB


Atenção

In [95]:
basketa= data2[['order_id','product_name_x']].value_counts().unstack().reset_index().fillna(0).set_index('order_id')
basketa.head()

product_name_x,0% Greek Strained Yogurt,1% Lowfat Milk,100 Calorie Per Bag Popcorn,100% Raw Coconut Water,100% Recycled Paper Towels,100% Whole Wheat Bread,2% Reduced Fat DHA Omega-3 Reduced Fat Milk,2% Reduced Fat Milk,2% Reduced Fat Organic Milk,85% Lean Ground Beef,...,Whole Milk,Whole Milk Plain Yogurt,Whole Milk Ricotta Cheese,Whole Wheat Bread,XL Emerald White Seedless Grapes,Yellow Bell Pepper,Yellow Onions,YoKids Blueberry & Strawberry/Vanilla Yogurt,"YoKids Squeezers Organic Low-Fat Yogurt, Strawberry","Yogurt, Strained Low-Fat, Coconut"
order_id,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [96]:
def encode_units(x):
    if x <= 0:
        return False
    if x >= 1:
        return True

basketa_sets = basketa.applymap(encode_units)

In [97]:
frequent_itemsetsz = apriori(basketa_sets, min_support=0.03, use_colnames=True,low_memory=True)
frequent_itemsetsz.sort_values('support',ascending=False)

,support,itemsets
2,0.181652,(Banana)
1,0.145827,(Bag of Organic Bananas)
17,0.102941,(Organic Strawberries)
8,0.094042,(Organic Baby Spinach)
14,0.083263,(Organic Hass Avocado)
7,0.068949,(Organic Avocado)
5,0.059461,(Large Lemon)
6,0.054854,(Limes)
22,0.054830,(Strawberries)
16,0.053458,(Organic Raspberries)


In [112]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.07)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Baby Spinach),(Avocado),0.115837,0.173100,0.032975,0.284668,1.644528,0.012924,1.155967
1,(Avocado),(Baby Spinach),0.173100,0.115837,0.032975,0.190497,1.644528,0.012924,1.092229
2,(Banana),(Avocado),0.383084,0.173100,0.084591,0.220815,1.275645,0.018279,1.061236
3,(Avocado),(Banana),0.173100,0.383084,0.084591,0.488679,1.275645,0.018279,1.206515
4,(Strawberries),(Avocado),0.182786,0.173100,0.040631,0.222287,1.284155,0.008991,1.063246
5,(Avocado),(Strawberries),0.173100,0.182786,0.040631,0.234725,1.284155,0.008991,1.067870
6,(Banana),(Baby Spinach),0.383084,0.115837,0.054407,0.142024,1.226072,0.010032,1.030522
7,(Baby Spinach),(Banana),0.115837,0.383084,0.054407,0.469689,1.226072,0.010032,1.163309
8,(Banana),(Blueberries),0.383084,0.076387,0.034264,0.089441,1.170898,0.005001,1.014337
9,(Blueberries),(Banana),0.076387,0.383084,0.034264,0.448552,1.170898,0.005001,1.118720


In [105]:
rules.sort_values('lift',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
17,(Strawberries),(Raspberries),0.182786,0.093030,0.033652,0.184104,1.978968,0.016647,1.111624
16,(Raspberries),(Strawberries),0.093030,0.182786,0.033652,0.361727,1.978968,0.016647,1.280352
1,(Avocado),(Baby Spinach),0.173100,0.115837,0.032975,0.190497,1.644528,0.012924,1.092229
0,(Baby Spinach),(Avocado),0.115837,0.173100,0.032975,0.284668,1.644528,0.012924,1.155967
14,(Banana),(Strawberries),0.383084,0.182786,0.090851,0.237156,1.297453,0.020828,1.071273
15,(Strawberries),(Banana),0.182786,0.383084,0.090851,0.497033,1.297453,0.020828,1.226555
5,(Avocado),(Strawberries),0.173100,0.182786,0.040631,0.234725,1.284155,0.008991,1.067870
4,(Strawberries),(Avocado),0.182786,0.173100,0.040631,0.222287,1.284155,0.008991,1.063246
3,(Avocado),(Banana),0.173100,0.383084,0.084591,0.488679,1.275645,0.018279,1.206515
2,(Banana),(Avocado),0.383084,0.173100,0.084591,0.220815,1.275645,0.018279,1.061236


In [114]:
rules.sort_values('lift',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
17,(Strawberries),(Raspberries),0.182786,0.093030,0.033652,0.184104,1.978968,0.016647,1.111624
16,(Raspberries),(Strawberries),0.093030,0.182786,0.033652,0.361727,1.978968,0.016647,1.280352
1,(Avocado),(Baby Spinach),0.173100,0.115837,0.032975,0.190497,1.644528,0.012924,1.092229
0,(Baby Spinach),(Avocado),0.115837,0.173100,0.032975,0.284668,1.644528,0.012924,1.155967
14,(Banana),(Strawberries),0.383084,0.182786,0.090851,0.237156,1.297453,0.020828,1.071273
15,(Strawberries),(Banana),0.182786,0.383084,0.090851,0.497033,1.297453,0.020828,1.226555
5,(Avocado),(Strawberries),0.173100,0.182786,0.040631,0.234725,1.284155,0.008991,1.067870
4,(Strawberries),(Avocado),0.182786,0.173100,0.040631,0.222287,1.284155,0.008991,1.063246
3,(Avocado),(Banana),0.173100,0.383084,0.084591,0.488679,1.275645,0.018279,1.206515
2,(Banana),(Avocado),0.383084,0.173100,0.084591,0.220815,1.275645,0.018279,1.061236
